In [2]:
from new_methods import *
import geopandas as gpd
import pandas as pd

bfe = gpd.read_file('bfe_mre_all_poc2.shp')
fsp = gpd.read_file('fsp_2.shp')

# Reproject files to UTM
bfe.to_crs(26913, inplace=True)
bfe = bfe[['ELEV', 'geometry']]

fsp.to_crs(26913, inplace=True)
fsp = fsp[['geometry']]
fsp.reset_index(inplace=True)

# Correct broken BFEs if there is any
bfe_bfe = bfe.sjoin(bfe, how='left')
bfe_bfe.reset_index(inplace=True)
brok = bfe_bfe.loc[bfe_bfe['index'] != bfe_bfe['index_right']]
if brok.shape[0] > 0:
    
    brok_geoms = brok[['index', 'geometry']]
    fix = brok.merge(brok_geoms, left_on='index_right', right_on='index')
    fix['new_geom'] = fix.apply(lambda x: linemerge(list((x.geometry_x, x.geometry_y))), axis=1)
    fix_gpd = gpd.GeoDataFrame(fix[['index_x', 'ELEV_left', 'new_geom']], geometry='new_geom', crs=26913)
    fix_gpd.rename(columns={'index_x': 'index', 'ELEV_left': 'ELEV', 'new_geom': 'geometry'}, inplace=True)
    fix_gpd.set_index('index', inplace=True)
    bfe_clean = bfe.loc[~bfe.index.isin(fix_gpd.index)]
    bfe = pd.concat([bfe_clean, fix_gpd])
    bfe = bfe.drop_duplicates('geometry')


# Extend BFEs over FSP Poly then reset bfe to new position
print('Extending BFE...')
bfe_ext = bfe_extend(bfe, fsp)

print('Clipping BFE...')
bfe = bfe_ext.overlay(fsp)

print('Splitting FSP...')
# Split FSP Poly by extended BFEs
fsp_s = split_fsp(fsp, bfe_ext)

# Remove slivers due to overlaps from extended BFEs
fsp_s = fsp_s.overlay(fsp)
fsp_s = fsp_s.loc[fsp_s.geom_type == 'Polygon']
fsp_s = fsp_s[['geometry']]
fsp_s.reset_index(inplace=True)




Extending BFE...
Clipping BFE...
Splitting FSP...


In [18]:
# Iterate through each FSP polygon
triangles = gpd.GeoDataFrame()
poly_errors = gpd.GeoSeries(crs=26913)
for i, f in fsp_s.iterrows():
    f = g(f.geometry, 26913)
    bfe_set = bfe.sjoin(f)
    
    bfe_set = bfe_set[['ELEV', 'geometry']]
    bfe_set = remove_multiline_BFE(bfe_set)
    
    print('POLY INDEX: ', i)
    #Ignoring Potential Polygon Slivers due to extension of BFEs overlapping small poritions of FSP
    if len(f.sjoin(bfe_set)) == 0:
        continue
    # Potential Island poly which will not intersect BFEs. Ignore!
    elif bfe_set.shape[0] == 0:
        continue
    else:
        try:
            # Getting Z-geom for BFE Points
            bfe_pts = bfe_zpts(bfe_set)

            # FSP Simplify, Interpolation, and Z-geom
            fsp_i_pts = fsp_pts(f, bfe_pts=bfe_pts, quadrats=4, diff_area=900)
            
            if fsp_i_pts is not None:
                fsp_i_pts = (fsp_i_pts.pipe(IDW, bfe=bfe_set, power=2)
                            .pipe(ELEV_2geom)
            )

                # Concat and Triangulate
                all_pts = pd.concat([bfe_pts, fsp_i_pts], ignore_index=True)
                all_pts_multigeom = MultiPoint(all_pts.geometry.to_list())

                tin = triangulate(all_pts_multigeom)
                tin_df = g(tin, 26913)

                # Extract Geom
                final_tin = extract_geom(tin_df)
                print('Unclipped: ', final_tin.shape)
                #final_tin = final_tin.loc[final_tin.geometry.centroid.within(f.geometry[0])]
                                        
                print('Clipped: ', final_tin.shape)
                triangles = pd.concat([triangles, final_tin], ignore_index=True)

            else:
                # Concat and Triangulate
                all_pts_multigeom = MultiPoint(bfe_pts.geometry.to_list())

                tin = triangulate(all_pts_multigeom)
                tin_df = g(tin, 26913)

                # Extract Geom
                final_tin = extract_geom(tin_df)
                print('Unclipped: ', final_tin.shape)
                #final_tin = final_tin.loc[final_tin.geometry.centroid.within(f.geometry[0])]
                                        
                print('Clipped: ', final_tin.shape)
                triangles = pd.concat([triangles, final_tin], ignore_index=True)
        except:
            print('Problem with Poly: ', i)
            p_er_dict = {i: f['geometry'][0]}
            error = gpd.GeoSeries(p_er_dict, crs=26913)
            poly_errors = pd.concat([poly_errors, error])
            continue
        
        
            


#final_tin.geometry.centroid.intersects(f.geometry[0] SAME AS CENTROID.WITHIN



POLY INDEX:  0
Unclipped:  (0, 1)
Clipped:  (0, 1)
POLY INDEX:  1
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  2
Unclipped:  (13, 13)
Clipped:  (13, 13)
POLY INDEX:  3
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  4
Unclipped:  (0, 1)
Clipped:  (0, 1)
POLY INDEX:  5
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  6
Unclipped:  (0, 1)
Clipped:  (0, 1)
POLY INDEX:  7
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  8
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  9
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  10
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  11
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  12
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  13
Unclipped:  (9, 13)
Clipped:  (9, 13)
POLY INDEX:  14
Unclipped:  (9, 13)
Clipped:  (9, 13)
POLY INDEX:  15
Unclipped:  (4, 13)
Clipped:  (4, 13)
POLY INDEX:  16
Unclipped:  (2, 13)
Clipped:  (2, 13)
POLY INDEX:  17
Unclipped:  (11, 13)
Clipped:  (11, 13)
POLY INDEX:  18
Unclipped:  (2, 13)
Clip

In [19]:
triangles.to_file('triangles.shp')

In [17]:
fsp_s.to_file('fsp_s.shp')